# Autoencoder

In [1]:
using Flux, Flux.Data.MNIST
using Flux: @epochs, onehotbatch, mse, throttle
using Base.Iterators: partition
using Juno: @progress
# using CuArrays

## Load data

In [2]:
# Encode MNIST images as compressed vectors that can later be decoded back into images.
imgs = MNIST.images();

In [3]:
# Partition into batches of size 1000
data = [float(hcat(vec.(imgs)...)) for imgs in partition(imgs, 1000)];

## Model

In [4]:
N = 32 # Size of the encoding

encoder = Dense(28^2, N, relu)
decoder = Dense(N, 28^2, relu)

m = Chain(encoder, decoder)

Chain(Dense(784, 32, NNlib.relu), Dense(32, 784, NNlib.relu))

## Loss function

In [5]:
loss(x) = mse(m(x), x)

loss (generic function with 1 method)

## Optimizer

Evaluation callback function

In [6]:
evalcb = throttle(() -> @show(loss(data[1])), 5)
opt = ADAM(params(m))

#43 (generic function with 1 method)

## Training

In [7]:
@epochs 10 Flux.train!(loss, zip(data), opt, cb=evalcb)

┌ Info: Epoch 1
└ @ Main /home/pika/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62


loss(data[1]) = 0.10237304323353962 (tracked)
loss(data[1]) = 0.06375317989259469 (tracked)


┌ Info: Epoch 2
└ @ Main /home/pika/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62


loss(data[1]) = 0.04933560076775887 (tracked)


┌ Info: Epoch 3
└ @ Main /home/pika/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62


...

loss(data[1]) = 0.02531252880059777 (tracked)
loss(data[1]) = 0.024370100492144043 (tracked)


┌ Info: Epoch 9
└ @ Main /home/pika/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62


loss(data[1]) = 0.02360503095753046 (tracked)


┌ Info: Epoch 10
└ @ Main /home/pika/.julia/packages/Flux/kJVo6/src/optimise/train.jl:62


loss(data[1]) = 0.023022113936505148 (tracked)


## Sample output

In [8]:
using Images

In [9]:
img(x::Vector) = Gray.(reshape(clamp.(x, 0, 1), 28, 28))

function sample()
  # 20 random digits
  before = [imgs[i] for i in rand(1:length(imgs), 20)]
  # Before and after images
  after = img.(map(x -> cpu(m)(float(vec(x))).data, before))
  # Stack them all together
  hcat(vcat.(before, after)...)
end

sample (generic function with 1 method)

In [10]:
save("sample.jpg", sample())